In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Creating a db

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///library.db

'Connected: @library.db'

# Creating tables

In [92]:
%%sql
CREATE TABLE User(
    userID INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL,
    age INTEGER NOT NULL,
    fines NUMERIC(5,2) DEFAULT 0,
    CHECK (age >= 7),
    CHECK (fines >= 0)
);

 * sqlite:///library.db
Done.


[]

In [93]:
%%sql
CREATE TABLE Librarian(
    librarianID INTEGER PRIMARY KEY,
    firstName VARCHAR(30) NOT NULL,
    lastName VARCHAR(30) NOT NULL,
    salary NUMERIC(6,2) DEFAULT 0,
    section VARCHAR(4) NOT NULL,
    CHECK (salary >= 0)
);

 * sqlite:///library.db
Done.


[]

In [94]:
%%sql
CREATE TABLE LibraryItem(
    libraryItemID INTEGER PRIMARY KEY,
    author REFERENCES Item(author),
    itemName REFERENCES Item(itemName) NOT NULL,
    toBeAdded BOOLEAN
);

 * sqlite:///library.db
Done.


[]

In [95]:
%%sql
CREATE TABLE Item(
    author VARCHAR(30),
    itemName VARCHAR(30) NOT NULL,
    type VARCHAR(15) NOT NULL,
    PRIMARY KEY (author, itemName)
);

 * sqlite:///library.db
Done.


[]

In [96]:
%%sql
CREATE TABLE BorrowedItem(
    userID REFERENCES User(userID) NOT NULL,
    libraryItemID REFERENCES LibraryItem(libraryItemID) NOT NULL,
    dueDate DATETIME DEFAULT (date(CURRENT_TIMESTAMP, '+14 day')) NOT NULL,
    PRIMARY KEY (userID, libraryItemID, dueDate)
);

 * sqlite:///library.db
Done.


[]

In [97]:
%%sql
CREATE TABLE Event(
    startTS DATETIME NOT NULL,
    endTS DATETIME NOT NULL,
    room VARCHAR(10) NOT NULL,
    eventName VARCHAR(30) NOT NULL,
    audience VARCHAR(20),
    maxRegistrations INTEGER DEFAULT 0,
    artist VARCHAR(30),
    author REFERENCES Item(author),
    itemName REFERENCES Item(itemName),
    PRIMARY KEY (startTS, room),
    CHECK (maxRegistrations >= 0)
);

 * sqlite:///library.db
Done.


[]

In [98]:
%%sql
CREATE TABLE EventRegistration(
    startTS REFERENCES Event(startTS) NOT NULL,
    room REFERENCES Event(room) NOT NULL,
    userID REFERENCES User(userID) NOT NULL,
    PRIMARY KEY (startTS, room)
);

 * sqlite:///library.db
Done.


[]

In [90]:
%%sql
DROP TABLE Item;
DROP TABLE User;
DROP TABLE Librarian;
DROP TABLE LibraryItem;
DROP TABLE BorrowedItem;
DROP TABLE Event;
DROP TABLE EventRegistration;

 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
(sqlite3.OperationalError) near "EventRegistration": syntax error
[SQL: EventRegistration;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [91]:
%%sql
DROP TABLE EventRegistration;

 * sqlite:///library.db
Done.


[]